In [2]:
import numpy as np # Linear algebra
import pandas as pd # data processing

In [3]:
test_df = pd.read_csv(r"E:\github material\toxic comment\test.csv")
train_df =  pd.read_csv(r"E:\github material\toxic comment\train.csv")
sample_submission =  pd.read_csv(r"E:\github material\toxic comment\sample_submission.csv")
test_labels =  pd.read_csv(r"E:\github material\toxic comment\test_labels.csv")

In [4]:
test_df.head()

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [5]:
train_df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [6]:
test_labels.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,-1,-1,-1,-1,-1,-1
1,0000247867823ef7,-1,-1,-1,-1,-1,-1
2,00013b17ad220c46,-1,-1,-1,-1,-1,-1
3,00017563c3f7919a,-1,-1,-1,-1,-1,-1
4,00017695ad8997eb,-1,-1,-1,-1,-1,-1


In [7]:
sample_submission.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.5,0.5,0.5,0.5,0.5,0.5
1,0000247867823ef7,0.5,0.5,0.5,0.5,0.5,0.5
2,00013b17ad220c46,0.5,0.5,0.5,0.5,0.5,0.5
3,00017563c3f7919a,0.5,0.5,0.5,0.5,0.5,0.5
4,00017695ad8997eb,0.5,0.5,0.5,0.5,0.5,0.5


# Data Preprocessing

In [8]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   id             159571 non-null  object
 1   comment_text   159571 non-null  object
 2   toxic          159571 non-null  int64 
 3   severe_toxic   159571 non-null  int64 
 4   obscene        159571 non-null  int64 
 5   threat         159571 non-null  int64 
 6   insult         159571 non-null  int64 
 7   identity_hate  159571 non-null  int64 
dtypes: int64(6), object(2)
memory usage: 9.7+ MB


In [9]:
labels =['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
for i in labels:
    print('The no of {} comments in file is {}' .format(i , train_df[i].sum()))

The no of toxic comments in file is 15294
The no of severe_toxic comments in file is 1595
The no of obscene comments in file is 8449
The no of threat comments in file is 478
The no of insult comments in file is 7877
The no of identity_hate comments in file is 1405


In [10]:
from string import punctuation
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from zipfile import ZipFile 
import tensorflow
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Embedding, Dropout, Activation, Input
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D
from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers
from sklearn.model_selection import train_test_split
%matplotlib inline

In [11]:
def data_cleaning(text):
    text = text.lower()
    text = ''.join([c for c in text if c not in punctuation])
    text = word_tokenize(text) #It actually returns the syllables from a single word
    stop_words =stopwords.words("english") #[stop word .word which have less imp. in meaning of sentence]
    text = [i for i in text if i not in stop_words ]
    text =[i for i in text if i.isalpha()] # check all the characters are alphabet 
    
    y= WordNetLemmatizer()
    normalized_text = []
    for word in text:
        normalized_word =y.lemmatize(word,'v') # every word convert in verb
        normalized_text.append(normalized_word)
    return normalized_text

In [12]:
data_1 = train_df[train_df['toxic'] ==1]
data_2 = train_df[train_df['toxic']==0][0:20000]
print(data_1.shape ,data_2.shape)

(15294, 8) (20000, 8)


In [13]:
sample_data= pd.concat([data_1 ,data_2] ,axis=0)

In [14]:
sample_data.sort_index(inplace=True)

In [15]:
sample_data

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
159494,fef4cf7ba0012866,"""\n\n our previous conversation \n\nyou fuckin...",1,0,1,0,1,1
159514,ff39a2895fc3b40e,YOU ARE A MISCHIEVIOUS PUBIC HAIR,1,0,0,0,1,0
159541,ffa33d3122b599d6,Your absurd edits \n\nYour absurd edits on gre...,1,0,1,0,1,0
159546,ffb47123b2d82762,"""\n\nHey listen don't you ever!!!! Delete my e...",1,0,0,0,1,0


In [16]:
sample_data['comment_text'] =sample_data['comment_text'].apply(data_cleaning)

In [17]:
length  = []
for i in sample_data['comment_text']:
   length.append(len(i))
sample_data['length'] =length
sample_data = sample_data[sample_data['length']>4]
sample_data=  sample_data[sample_data['length']<500]
sample_data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,length
0,0000997932d777bf,"[explanation, edit, make, username, hardcore, ...",0,0,0,0,0,0,26
1,000103f0d9cfb60f,"[daww, match, background, colour, im, seemingl...",0,0,0,0,0,0,11
2,000113f07ec002fd,"[hey, man, im, really, try, edit, war, guy, co...",0,0,0,0,0,0,22
3,0001b41b1c6bb37e,"[cant, make, real, suggestions, improvement, w...",0,0,0,0,0,0,49
4,0001d958c54c6e35,"[sir, hero, chance, remember, page, thats]",0,0,0,0,0,0,6


In [18]:
max_length =200
max_features =20000
def data_preprocessing(data ,max_features=20000,max_len=200):
    tokenizer = Tokenizer(num_words = max_features)
    tokenizer.fit_on_texts(sample_data['comment_text'])
    data = tokenizer.texts_to_sequences(data)
    data = pad_sequences(data, max_len)
    return data

In [19]:
labels = ['toxic', 'severe_toxic' ,'obscene', 'threat','insult' ,'identity_hate']
final_data =pd.DataFrame(data_preprocessing(sample_data['comment_text']))
final_data['index'] = sample_data.index
final_data.set_index("index",inplace=True)
final_data[labels]=sample_data[labels]
final_data.head()

,0,1,2,3,4,5,6,7,8,9,...,196,197,198,199,toxic,severe_toxic,obscene,threat,insult,identity_hate
index,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,0,0,0,...,2,89,19,2509,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,25,9,1105,137,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,202,856,606,407,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,74,415,389,952,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,775,488,2,91,0,0,0,0,0,0


In [20]:
labels=['toxic','severe_toxic','obscene','threat','insult','identity_hate']
for i in labels:
    print('The no of {} comments in file is {}'.format(i,final_data[i].sum()))

The no of toxic comments in file is 12816
The no of severe_toxic comments in file is 1252
The no of obscene comments in file is 6507
The no of threat comments in file is 391
The no of insult comments in file is 6106
The no of identity_hate comments in file is 1109


In [21]:
toxic_a = final_data[final_data['toxic']==1]
toxic_b = final_data[final_data['toxic']==0][:20000]
toxic_data = pd.concat((toxic_a,toxic_b),axis=0).sort_index()

severe_a=final_data[final_data['severe_toxic']==1]
severe_b=final_data[final_data['severe_toxic']==0][:2000]
severe_data=pd.concat((severe_a,severe_b),axis=0).sort_index()

obscene_a=final_data[final_data['obscene']==1]
obscene_b=final_data[final_data['obscene']==0][:8000]
obscene_data=pd.concat((severe_a,severe_b),axis=0).sort_index()

threat_a=final_data[final_data['threat']==1]
threat_b=final_data[final_data['threat']==0][:600]
threat_data=pd.concat((threat_a,threat_b),axis=0).sort_index()


insult_a=final_data[final_data['insult']==1]
insult_b=final_data[final_data['insult']==0][:8000]
insult_data=pd.concat((threat_a,threat_b),axis=0).sort_index()


hate_a=final_data[final_data['identity_hate']==1]
hate_b=final_data[final_data['identity_hate']==0][:1500]
hate_data=pd.concat((hate_a,hate_b),axis=0).sort_index()

In [22]:
toxic_data

,0,1,2,3,4,5,6,7,8,9,...,196,197,198,199,toxic,severe_toxic,obscene,threat,insult,identity_hate
index,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,0,0,0,...,2,89,19,2509,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,25,9,1105,137,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,202,856,606,407,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,74,415,389,952,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,775,488,2,91,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159478,0,0,0,0,0,0,0,0,0,0,...,102,80,410,70,1,0,0,0,0,0
159494,0,0,0,0,0,0,0,0,0,0,...,247,13,3,170,1,0,1,0,1,1
159541,0,0,0,0,0,0,0,0,0,0,...,212,601,24,2607,1,0,1,0,1,0


In [23]:
test_df['comment_text']=test_df['comment_text'].apply(data_cleaning)

In [24]:
test_df=pd.DataFrame(data_preprocessing(test_df['comment_text']))
test_df.head()

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,0,0,0,0,0,0,0,0,0,0,...,12671,6,1802,423,1598,10,140,54,359,22
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1236,268,582,2757
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,23
3,0,0,0,0,0,0,0,0,0,0,...,389,362,23,2250,719,634,719,52,25,159
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,7,8235,5,1


In [25]:
test_df.to_csv('processed_test_data.csv',index=False)

In [26]:
pd.read_csv('processed_test_data.csv')

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,0,0,0,0,0,0,0,0,0,0,...,12671,6,1802,423,1598,10,140,54,359,22
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1236,268,582,2757
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,23
3,0,0,0,0,0,0,0,0,0,0,...,389,362,23,2250,719,634,719,52,25,159
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,7,8235,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153159,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,697,163,334,117
153160,0,0,0,0,0,0,0,0,0,0,...,3708,925,823,197,774,13842,2242,128,1424,2870
153161,0,0,0,0,0,0,0,0,0,0,...,809,1555,1535,109,3729,2459,3860,4621,809,367
153162,0,0,0,0,0,0,0,0,0,0,...,687,8746,342,166,179,838,75,32,9500,3881


# Training and prediction

In [27]:
class lstm_model(object):
    def __init__(self,max_features=20000):
        self.max_features=max_features
        self.model=Sequential()
        self.model.add(Input(shape=max_length,))
        self.model.add(Embedding(self.max_features,128))
        self.model.add(Bidirectional(LSTM(50, return_sequences=True,recurrent_dropout=0.2)))
        self.model.add(Bidirectional(LSTM(50, return_sequences=True,recurrent_dropout=0.2)))
        self.model.add(GlobalMaxPool1D())
        self.model.add(Dense(1,activation='sigmoid'))
        self.model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    
    def fit(self,X,y,batchsize,epochs):

        self.model.fit(X,y, batch_size=batchsize, epochs=epochs, validation_split=0.2)
        
    def predict(self,data):
        return self.model.predict(data)
        
    
    def predict_classes(self,data):
        return self.model.predict_classes(data)
        

In [28]:
toxic_model=lstm_model()
toxic_model.fit(X=toxic_data.drop(labels,axis=1), y=toxic_data['toxic'],batchsize=100,epochs=1)

247/247 [==============================] - 1183s 5s/step - loss: 0.3346 - accuracy: 0.8544 - val_loss: 0.3769 - val_accuracy: 0.8366


In [30]:
toxic_predictions=toxic_model.predict_classes(test_df)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [31]:
sample_submission['toxic']=toxic_predictions

In [32]:
sample_submission

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,1,0.5,0.5,0.5,0.5,0.5
1,0000247867823ef7,0,0.5,0.5,0.5,0.5,0.5
2,00013b17ad220c46,0,0.5,0.5,0.5,0.5,0.5
3,00017563c3f7919a,0,0.5,0.5,0.5,0.5,0.5
4,00017695ad8997eb,0,0.5,0.5,0.5,0.5,0.5
...,...,...,...,...,...,...,...
153159,fffcd0960ee309b5,0,0.5,0.5,0.5,0.5,0.5
153160,fffd7a9a6eb32c16,0,0.5,0.5,0.5,0.5,0.5
153161,fffda9e8d6fafa9e,0,0.5,0.5,0.5,0.5,0.5
153162,fffe8f1340a79fc2,0,0.5,0.5,0.5,0.5,0.5


In [36]:
obscene_model=lstm_model()
obscene_model.fit(X=obscene_data.drop(labels,axis=1), y=obscene_data['obscene'],batchsize=100,epochs=1)


27/27 [==============================] - 128s 5s/step - loss: 0.5745 - accuracy: 0.7497 - val_loss: 1.1834 - val_accuracy: 0.0568


In [37]:
obscene_predictions=obscene_model.predict_classes(test_df)

In [38]:
threat_model=lstm_model()
threat_model.fit(X=threat_data.drop(labels,axis=1), y=threat_data['threat'],batchsize=100,epochs=1)


8/8 [==============================] - 34s 4s/step - loss: 0.6045 - accuracy: 0.7576 - val_loss: 1.5412 - val_accuracy: 0.0000e+00


In [39]:
threat_predictions=threat_model.predict_classes(test_df)

In [40]:
insult_model=lstm_model()
insult_model.fit(X=insult_data.drop(labels,axis=1), y=insult_data['insult'],batchsize=100,epochs=1)


8/8 [==============================] - 32s 4s/step - loss: 0.6137 - accuracy: 0.7538 - val_loss: 1.0995 - val_accuracy: 0.2663


In [41]:
insult_predictions=insult_model.predict_classes(test_df)

In [42]:
hate_model=lstm_model()
hate_model.fit(X=hate_data.drop(labels,axis=1), y=hate_data['identity_hate'],batchsize=100,epochs=1)


21/21 [==============================] - 97s 5s/step - loss: 0.6236 - accuracy: 0.7173 - val_loss: 1.0124 - val_accuracy: 0.0000e+00


In [43]:
hate_predictions=hate_model.predict_classes(test_df)

In [44]:
severe_model=lstm_model()
severe_model.fit(X=severe_data.drop(labels,axis=1),y=severe_data['severe_toxic'],batchsize=100,epochs=1)


27/27 [==============================] - 126s 5s/step - loss: 0.5587 - accuracy: 0.7447 - val_loss: 1.2845 - val_accuracy: 0.0031


In [45]:
severe_predictions=severe_model.predict_classes(test_df)

In [48]:
sample_submission['toxic']=toxic_predictions
sample_submission['severe_toxic'] = severe_predictions
sample_submission['obscene'] = obscene_predictions
sample_submission['threat'] = threat_predictions
sample_submission['identity_hate'] = hate_predictions
sample_submission['insult']=insult_predictions
sample_submission.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,1,0,0,0,0,0
1,0000247867823ef7,0,0,0,0,0,0
2,00013b17ad220c46,0,0,0,0,0,0
3,00017563c3f7919a,0,0,0,0,0,0
4,00017695ad8997eb,0,0,0,0,0,0
